# Лабораторная работа №1

## Задания

Самостоятельно написать код, реализующий искусственный нейрон с сигма-функцией активации, и возможность строить на его основе многослойные сети. Код должен также реализовывать градиентный спуск и обратное распространение ошибки.

На основе вашего кода:

1. Решить задачу  классификации датаcета Iris одним нейроном.
2. Решить задачу  классификации датаcета Iris одним  нейросетью из 2 слоев по 10 нейронов в слое.
3. Отрисовать разделяющую линию для обеих моделей. Сравнить метрики классификации.

## Реализация

### Библиотеки

In [1]:
from sklearn.datasets import load_iris
import numpy as np

### Сигма-функция

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def der_sigmoid(x):  # производная сигма-функции
    return sigmoid(x) * (1 - sigmoid(x))

### Нейрон

Его формула: `y = f(sum_i(w_i*x_i + b_i))`, где:

* w - веса
* x - входные данные
* b - сдвиг
* f - функция активации
* i - индекс
* sum_i - сумма по индексам

In [3]:
class Neuron:
    def __init__(self, input_count, default_weight, default_bias):
        self.attribute_count = input_count  # кол-во весов и сдвигов равно кол-во вводных данных
        self.weights = np.full((1, self.attribute_count), default_weight)  # вектор весов нейрона, при инициализации задаётся значение по умолчанию
        self.bias = np.full((1, self.attribute_count), default_bias)  # вектор сдвигов нейрона, при инициализации задаётся значение по умолчанию
        self.sum_result = 0
        
    def output(self, input_values):
        self.sum_result = np.sum(np.dot(self.weights, input_values) + self.bias)
        return sigmoid(self.sum_result)

### Функция ошибки

Здесь будет использоваться MSE (Mean Square Error)

Её формула: `sum_i((true_i - pred_i)^2) / N`, где:

* true - истинные значения
* pred - предсказанные значения
* i - индекс
* N - размер
* sum_i - сумма по индексам

In [4]:
def mse(y_true, y_pred):
    return ((y_true - y_pred)**2).mean()

def der_mse(y_true, y_pred):  # производная функции ошибок (для одного предсказанного значения y_pred)
    return -2 * (y_true - y_pred)

### Слой нейронной сети

In [5]:
class Layer:
    def __init__(self, neuron_count):
        self.neuron_count = neuron_count
        self.neurons = []
        self.input_count = 0
        self.input_values = None

    def setup_neurons(self, input_count, default_weight=1, default_bias=0):  # изначальный вес = 1, изначальный свдиг = 0, если не задано иное
        if input_count <= 0:
            raise ValueError("No inputs to this neuron were provided")

        self.input_count = input_count
        
        for i in range(self.neuron_count):
            new_neuron = Neuron(self.input_count, default_weight, default_bias) 
            self.neurons.append(new_neuron)

    def input_output(self, input_values):
        if len(input_values) != self.input_count:
            raise ValueError("The amount of input values doesn't match the amount of inputs for this layer's neurons")

        self.input_values = input_values

        outputs = []

        for neuron in self.neurons:
            outputs.append(neuron.output(input_values))

        return np.array(outputs)

### Нейронная сеть

In [6]:
class NeuralNetwork:
    def __init__(self, layers=[]):
        self.layers = layers
        self.layer_count = len(self.layers)
        self.attribute_count = 0  # кол-во вводимых аттрибутов
        self.verbose = False

    def add(self, layer):
        self.layers.append(layer)
        self.layer_count += 1

    def log(self, text):
        if self.verbose:
            print(text)

    def gradient_descent(self, target_neuron, target_layer, cur_neuron, cur_layer):  # рекурсивно считаем градиентный спуск
        layer = self.layers[cur_layer]
        neuron = layer.neurons[cur_neuron] 
        der = der_sigmoid(neuron.sum_result)  # значение производной от функции активации этого нейрона

        if cur_layer == target_layer and cur_neuron == target_neuron:
            return der * layer.input_values, np.fill((1, neuron.attribute_count), der)  # искомый нейрон, производная по весам будет равна вводным данным, а по сдвигам равна 1

        if cur_layer - 1 == target_layer:  # если на предыдущем слое находится искомый нейрон, то сразу берём его значение
            prev_weight, prev_bias = self.gradient_descent(target_neuron, target_layer, target_neuron, target_layer)
            
            target_weight = neuron.weights[target_neuron]  # вес, относящийся к искомому нейрону, остальные веса не нужны, т.к. производные по нейронам, к которым они относятся, будут равны 0
            
            return der * target_weight * prev_weight, der * target_weight

        weight, bias = np.zeros((1, neuron.attribute_count)), np.zeros((1, neuron.attribute_count))
        
        for prev_neuron in range(self.layers[cur_layer - 1]):  # если же на предыдущем слое нету искомого нейрона, то суммируем все произведения весов на производные
            prev_weight, prev_bias = self.gradient_descent(target_neuron, target_layer, prev_neuron, cur_layer - 1)

            neuron_weight = neuron.weights[prev_neuron]  # вес, относящийся к предыдущему нейрону

            weight += prev_weight * neuron_weight
            bias += prev_bias * neuron_weight
            
        return der * weight, der * bias
        
    def fit(self, X_train, y_train, X_test, y_test, learning_speed=0.1, epochs=200, early_stopping=None, verbose=False):
        if len(X_train) != len(y_train) or len(X_test) != len(y_test):
            raise ValueError("X and y must be the same size")
            
        if self.layers[-1].neuron_count != 1:
            raise ValueError("No dense layer at the end was provided")

        if learning_speed <= 0:
            raise ValueError("Learning speed must be more than 0")

        self.attribute_count = len(X_train[0])
        self.verbose = verbose

        # Инициализация каждого слоя

        for layer in range(self.layer_count):
            self.log(f"Initialising layer {layer+1} out of {self.layer_count}")
            
            if layer == 0:
                self.layers[layer].setup_neurons(input_count=self.attribute_count)  # первый слой - входной, у него кол-во входных данных равно кол-ву вводимых атрибутов

            else:
                self.layers[layer].setup_neurons(input_count=self.layers[layer-1].neutron_count)  # у каждого последующего слоя кол-во входных данных равно кол-ву нейронов на предыдущем слое

        train_value_count = len(X_train)

        # Начинаем обучение

        last_error = None

        for epoch in range(epochs):
            self.log(f"Epoch {epoch+1} out of {epochs}")
            y_pred = []  # предсказанные значения
            
            for value in range(train_value_count):
                self.log(f"Predicting for train value {value+1} out of {train_value_count}")
                
                neuron_output = X_train[value]

                for layer in range(self.layer_count):
                    self.log(f"Getting input/output for layer {layer+1} out of {self.layer_count}")
                    
                    neuron_output = self.layers[layer].input_output(neuron_output)

                y_pred.append(neuron_output)  # значение последнего слоя и будет предсказанным значением

            y_pred = np.array(y_pred)  # получили предсказание
            error = mse(y_train, y_pred)  # значение ошибки
            self.log(f"Predicting done, current error is {error}")

            if early_stopping is not None:  # early_stopping - это минимальная разница в ошибке, после которой можно остановиться
                if last_error is not None:
                    if abs(error - last_error) < early_stopping:
                        break
                
                last_error = error

            # Теперь выполняем градиентный спуск

            last_neuron, last_layer = 1, self.layer_count-1  # градиентный спуск идёт с конца

            for value in range(train_value_count):
                der_loss = der_mse(y_train[value], y_pred[value])  # производная функции ошибок
                
                for layer in range(self.layer_count):
                    cur_layer = self.layers[layer]
                    
                    for neuron in range(cur_layer.neuron_count):
                        cur_neuron = cur_layer.neurons[neuron]
                        
                        self.log(f"Gradient descent for NEURON {neuron+1} out of {cur_layer.neuron_count} for LAYER {layer+1} out of {self.layer_count} for VALUE {value+1} out of {train_value_count}")
    
                        weight_descent, bias_descent = self.gradient_descent(neuron, layer, last_neuron, last_layer)
                        
                        cur_neuron.weights -= weight_descent * learning_speed * der_loss  # изменение весов нейрона по результатам спуска
                        cur_neuron.bias -= bias_descent * learning_speed * der_loss  # изменение сдвигов нейрона по результатам спуска

        # Теперь начинаем предсказание

        y_pred = []  # предсказанные значения
        test_value_count = len(X_test)

        for value in range(test_value_count):
            self.log(f"Predicting for test value {value+1} out of {test_value_count}")
            
            neuron_output = X_test[value]

            for layer in range(self.layer_count):
                self.log(f"Getting input/output for layer {layer+1} out of {self.layer_count}")
                
                neuron_output = self.layers[layer].input_output(neuron_output)

            y_pred.append(neuron_output)  # значение последнего слоя и будет предсказанным значением

        return np.array(y_pred)  # получили предсказание

Загрузка датасета

In [7]:
df = load_iris()
X, y = df["data"], df["target"]

### Деление датасета

In [8]:
def train_test_split(X, y, split=0.2):
    if split > 1 or split < 0:
        raise ValueError("train_test split must be within [0, 1]")
    
    split_index = int(len(X)*split)
    return np.array(X[split_index:]), np.array(y[split_index:]), np.array(X[:split_index]), np.array(y[:split_index])

In [9]:
X_train, y_train, X_test, y_test = train_test_split(X, y)